In [31]:
import os
import gradio as gr

from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
print(openai_api_key[:8])
gpt = OpenAI()

context = {}

employees = Path(".").glob("knowledge-base/employees/*.md")
for employee in employees:
    name = employee.name.split(' ')[-1][:-3].lower()
    with open(employee, "r", encoding="utf-8") as f:
        text = f.read()
        context[name] = text

products = Path(".").glob("knowledge-base/products/*.md")
for product in products:
    name = product.name[:-3].lower()
    with open(product, "r", encoding="utf-8") as f:
        text = f.read()
        context[name] = text

common_data = Path(".").glob("knowledge-base/company/*.md")
for data in common_data:
    text = ''
    with open(data, "r", encoding="utf-8") as f:
        text += f.read()
        context["insurellm"] = text
    
                        


sk-proj-


In [32]:
context.keys()

dict_keys(['chen', 'spencer', 'tran', 'blake', 'lancaster', 'thompson', 'greene', 'thomson', 'trenton', 'harper', 'bishop', 'carter', 'rellm', 'markellm', 'homellm', 'carllm', 'insurellm'])

In [38]:
systemPrompt = ("You are an expert in answering accurate questions about Insurellm, The Insurance Tech Company."
    "Give brief accurate answers. If you don't know the answer, say so."
    "Do not make anything up if you haven't been provided with relevant content."               
    )

def add_relevant_content(message):
    content = []
    for key, value in context.items():
        lowered = message.lower()
        if key in lowered:
            content.append(value)

    if content:
        message += f"\nHere is content that could be relevant for answering the question:\n"
        for data in content:
            message += data+"\n"
        
    return message


def chat(message, history):
    messages = [{"role": "system", "content": systemPrompt}]
    messages += [{"role": it["role"], "content": it["content"]} for it in history]
    message = add_relevant_content(message)
    messages += [{"role": "user", "content": message}]

    prompt_messages = [
        {"role": "system", "content": systemPrompt},
        {"role": "user", "content": message}
    ]
    
    stream = gpt.chat.completions.create(
        model="gpt-5-mini",
        messages=messages,
        stream=True
    )

    resp = ''
    for chunk in stream:
        text = chunk.choices[0].delta.content or ''
        resp += text
        yield resp
        


gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
